In [5]:
%load_ext autoreload
%autoreload 2

# 🎮 Discord


In [2]:
import os
import discord  # type: ignore

intents = discord.Intents.default()
intents.message_content = True
client = discord.Client(intents=intents) 
await client.login(os.getenv("DISCORD_BOT_TOKEN"))

In [7]:
my_id = 304721494276308992
user: discord.User = await client.fetch_user(my_id)
print(user._to_minimal_user_json())

{'username': 'ambratolm', 'id': 304721494276308992, 'avatar': 'ad7d844a8ef9b1dc10bc9570e4bb328e', 'discriminator': '0', 'global_name': '𝘼𝙢𝙗𝙧𝙖𝙩𝙤𝙡𝙢', 'bot': False}


In [27]:
import csv
from io import StringIO
from discord import TextChannel
async def get_recent_channel_users(channel: TextChannel, limit=3) -> list:
    """Fetch recent unique message authors from a text channel."""
    users = set()  # Use a set to store unique users
    async for msg in channel.history(limit=limit):
        users.add(msg.author)  # Add author to the set (duplicates are ignored)
    return list(users)  # Convert the set back to a list

async def get_recent_channel_users_csv(channel: TextChannel, limit=10) -> str:
    """Fetch recent unique message authors from a text channel in CSV format.
    """
    users = set()
    async for msg in channel.history(limit=limit):
        user_info = {  # Store in a dictionary for CSV writer
            "id": msg.author.id,
            "name": msg.author.name,
            "display_name": msg.author.display_name,
        }
        users.add(tuple(user_info.values()))  # Stringify the user object directly
    # Create CSV string in memory:
    output = StringIO()  # Use StringIO for in-memory file-like object
    writer = csv.DictWriter(output, fieldnames=user_info.keys())  # Use DictWriter
    writer.writeheader()  # Write the header row
    for user_data in users:
        writer.writerow(dict(zip(user_info.keys(), user_data)))  # Write each user as row
    return output.getvalue() 


guild_id = 1337479592398553121  # C-T-S guild id
channel_id = 1337479593183023226  # general channel id
guild = await client.fetch_guild(guild_id)
channel = await guild.fetch_channel(channel_id)

await get_recent_channel_users_csv(channel, 10)

'id,name,display_name\r\n1330783368773632052,𝓐𝓒𝓣𝓲𝓿𝓪,𝓐𝓒𝓣𝓲𝓿𝓪\r\n304721494276308992,ambratolm,𝘼𝙢𝙗𝙧𝙖𝙩𝙤𝙡𝙢\r\n'

In [ ]:
import logging
import os
from discord import Guild  # type: ignore
from db.main import *
from db.model import *
from db.actor import *
db_uri = os.getenv("MONGO_DB_URI")
db_client = ActDbClient(host=db_uri, name="ACT")
db = db_client.get_database_by_id(guild_id)

⏳ [db.main] Database client opening...
✅ [db.main] 🍃 Database client connected to localhost:1717.
ℹ️ [db.main] 
┌─────────────┐
│ Databases:  │
│ • ACT       │
│ • ACT_c-t-s │
│ • admin     │
│ • config    │
│ • local     │
└─────────────┘


Collection(Database(MongoClient(host=['localhost:1717'], document_class=dict, tz_aware=False, connect=True), 'ACT_c-t-s'), 'ai_chat_users')


In [66]:
member = await guild.fetch_member(304721494276308992)


async def save_member(member):
    ai_chat_users = db["ai_chat_users"]
    return ai_chat_users.update_one(
        {"_id": member.id },
        {
            "$set": {
                "_id": member.id,
                "name": member.name,
                "display_name": member.display_name,
            }
        },
        upsert=True,
    )

def load_members_csv():
    ai_chat_users = db["ai_chat_users"]
    members = ai_chat_users.find()
    output = StringIO()
    writer = csv.DictWriter(output, fieldnames=["_id", "name", "display_name"] )
    writer.writeheader()
    for member in members:
        writer.writerow(member)
    return output.getvalue() 


await save_member(member)
load_members_csv()

'_id,name,display_name\r\n998345861585186836,c_o_m_o_n,Collins\r\n304721494276308992,ambratolm,𝘼𝙢𝙗𝙧𝙖𝙩𝙤𝙡𝙢\r\n'